Русский LLM-помощник (saiga) с кэшем, используя RAG (Retrieval-Augmented Generation)    
https://habr.com/ru/articles/769124/

https://github.com/akocherovskiy/LLM_chat-bot_RAG_approach

In [1]:
# !pip install peft transformers langchain sentencepiece

In [2]:
import torch

In [3]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [4]:
assert torch.cuda.is_available()
display( torch.cuda.get_device_name() )
device = torch.device('cuda:0')

'Tesla P40'

In [5]:
adapt_model_name = 'IlyaGusev/saiga_mistral_7b_lora'
base_model_name = 'Open-Orca/Mistral-7B-OpenOrca'

In [6]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoPeftModelForCausalLM.from_pretrained(
        adapt_model_name, 
        torch_dtype=torch.bfloat16
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = model.to(device)    

In [7]:
info = (
'Небольшой свечной завод, расположенный по адресу ул. Ленина 22, '
'находится в городе Градск. Доехать до завода от центрального автовокзала '
'можно на автобусе номер 666. Завод специализируется на производстве свечей '
'различных размеров и форм. Открытая вакансия: Производственный работник - '
'зарплата 150 тысяч рублей в месяц. Требования: 6 разряд свечника, 25 лет опыта. '
'Претендент на работу должен заполнить анкету, предоставить резюме, пройти собеседование '
'с представителями компании. Для работников завода предусмотрены следующие бонусы: '
'Медицинское страхование, Абонемент в бассейн. Завод активно проводит праздничные мероприятия и игры. '
)


In [8]:
template = info+'{question}\nbot: Вот ответ на ваш вопрос длиной не более 10 слов:'

In [9]:
def get_answer(info_prompt, question,device=device):
    prompt = info_prompt.format(question=question)
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(
            input_ids=inputs["input_ids"].to(device),
            top_p=0.5,
            temperature=0.3,
            attention_mask=inputs["attention_mask"],
            max_new_tokens=100,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True
        )

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    parsed_answer = output.split("Вот ответ на ваш вопрос длиной не более 10 слов:")[1].strip()
    if 'bot:' in parsed_answer: parsed_answer = parsed_answer.split("bot:")[0].strip()
    return parsed_answer

In [10]:
questions = [
"Какой адрес вашей компании?",
"Где находится офис вашей компании?",
"Где точно находится офис вашей компании?",
# "Как добраться до офиса вашей компании?",
"Что производит ваша компания?",
"Какую продукцию производит ваша компания?",
"Что именно производит ваша компания?",
#"Какие работники вам нужны?",
"Какие должности вы ищете?",
"Какие работники вам нужны в данный момент?",
"Какие специалисты вам требуются?",
#"Какие специалисты вам необходимы?",
"Какими навыками и умениями должен обладать специалист?",
#"Какие компетенции и знания необходимы специалисту?",
"Какими навыками и качествами должен владеть специалист?",
#"Какие специфические умения и опыт требуются у специалиста?",
# "Сколько платят на этой должности?",
# "Какова зарплата для этой должности?",
# "Какая заработная плата предлагается?",
# "Какие условия оплаты?",
# "Какие варианты и размеры заработной платы доступны для этой должности?",
# "Какая зарплата ожидается для новых сотрудников на этой позиции?",
"Как можно устроиться к вам на работу?",
# "Как устроиться к вам на работу?"
#"Какой процесс приема на работу?",
# "Как я могу подать заявку на работу к вам в компанию?",
# "Какие шаги мне нужно предпринять, чтобы устроиться к вам на завод?",
# "Что сделать, чтобы устроиться к вам на завод?",
# "Какие дополнительные бонусы и привилегии предоставляются?",
# "Какие бонусы и вознаграждения доступны для сотрудников?",
# "Какие бонусные программы предоставляются?",
# "Какие дополнительные бонусы и предлагаются?"
]

In [11]:
from langchain.prompts import PromptTemplate
info_prompt = PromptTemplate.from_template(template)

In [12]:
import time

In [13]:
# Find the index of the maximum value
for q in questions:
    print(q)
    start_time = time.time()
    answer = get_answer(info_prompt, q)
    time_s = ("[time: %.1f seconds]" % (time.time() - start_time))
    print(f'MODEL: {answer}   {time_s}\n\n')

Какой адрес вашей компании?
MODEL: ул. Ленина 22, Градск.   [time: 7.9 seconds]


Где находится офис вашей компании?
MODEL: ул. Ленина 22, Градск.   [time: 7.5 seconds]


Где точно находится офис вашей компании?
MODEL: ул. Ленина 22, Градск.   [time: 7.5 seconds]


Что производит ваша компания?
MODEL: свечи различных размеров и форм.   [time: 7.5 seconds]


Какую продукцию производит ваша компания?
MODEL: свечи разных размеров и форм.   [time: 7.5 seconds]


Что именно производит ваша компания?
MODEL: свечи разных размеров и форм.   [time: 7.5 seconds]


Какие должности вы ищете?
MODEL: Производственный работник.   [time: 7.5 seconds]


Какие работники вам нужны в данный момент?
MODEL: Производственный работник.   [time: 7.5 seconds]


Какие специалисты вам требуются?
MODEL: Продавец свечей, мастер по ремонту свечей, продавец свечей.   [time: 7.5 seconds]


Какими навыками и умениями должен обладать специалист?
MODEL: Свечник, 6 разряд, 25 лет опыта.   [time: 7.5 seconds]


Какими навы